In [1]:
!pip install -Uq transformers bitsandbytes datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!nvidia-smi

Thu Dec  5 04:54:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [4]:
model_name = "gpt2-large"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name, pad_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
text="안녕하세요, 제 이름은 임경태입니다. 오늘 수업의 주제는"
token = tokenizer(text,return_tensors='pt').to(model.device)
output = model.generate(**token, max_new_tokens=128)
print(tokenizer.decode(output[0]))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


안녕하세요, 제 이름은 임경태입니다. 오늘 수업의 주제는 이름은 임경태입니다. 제 이름은 임경태입니다. 제 이름은 임경태입니다. 제 이름은 임경태입니다. 제 이름은 임경�


In [ ]:
ds = load_dataset("PerRing/CulturaX_ko_10k",split='train')
ds = ds.shuffle(42)

README.md:   0%|          | 0.00/294 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/29.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
ds[0]

{'text': "안녕하세요, 웨일을 너무나도 잘 사용하고 있는 점에서 일단 감사드립니다^^ 제가 8월 2주차 LG그램 노트북 이벤트에 당첨 되었는데 배송이 언제쯤 오나 궁금하여 문의를 드립니다. 'dlems0818'이라는 아이디 입니다. 메일이 와서 개인정보를 입력하고 나면 확인이라고 떠서 제대로 전달되었는지 걱정이 됩니다.ㅠㅠ 그런데 아직 배송이 안 와서 조마조마 하고 있습니다.ㅎㅎ 상품이 고가인데 설마 전달이 안 됬을까봐... 그래서 바쁘시지만 간단한 답변이라도 달아주셨으면 합니다. 언제쯤 배송이 올까요??? 그리고 항상 웨일을 좋은 쪽으로 발전시키려 하시는 모습들이 정말 감사하고 잘 쓰도록 하겠습니다!!!"}

In [ ]:
def tokenize_function(example):
    return tokenizer(example['text'],max_length=1024,truncation=True)
ds = ds.map(tokenize_function)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
ds

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

In [ ]:
collate_fn = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
collate_fn(ds[:3]['input_ids'])

{'input_ids': tensor([[  168,   243,   230,  ..., 50257, 50257, 50257],
         [  167,    99,   105,  ...,   222,   168,   246],
         [  167,   100,   234,  ..., 50257, 50257, 50257]]),
 'labels': tensor([[ 168,  243,  230,  ..., -100, -100, -100],
         [ 167,   99,  105,  ...,  222,  168,  246],
         [ 167,  100,  234,  ..., -100, -100, -100]])}

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    report_to="none",
    optim="adamw_bnb_8bit",
    gradient_checkpointing=True,
    max_steps=50,
    logging_steps=5,
)

In [ ]:
trainer = Trainer(
        model=model,
        data_collator=collate_fn,
        args=training_args,
        train_dataset=ds,
    )

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
5,2.262500
10,2.043800
15,1.906200
20,1.814100
25,1.717200
30,1.989100
35,1.707800
40,1.843800
45,1.895300
50,1.779700


TrainOutput(global_step=50, training_loss=1.8959375, metrics={'train_runtime': 169.6068, 'train_samples_per_second': 0.295, 'train_steps_per_second': 0.295, 'total_flos': 184163085849600.0, 'train_loss': 1.8959375, 'epoch': 0.005})

In [ ]:
text="안녕하세요, 제 이름은 임경태입니다. 오늘 수업의 주제는"
token = tokenizer(text,return_tensors='pt').to(model.device)
output = model.generate(**token, max_new_tokens=128)
print(tokenizer.decode(output[0]))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


안녕하세요, 제 이름은 임경태입니다. 오늘 수업의 주제는 이름은 임경태입니다. 오늘 수업의 주제는 이름은 임경태입니다. 오늘 수업의 주제는 이름은 임경태입니다. 오늘 �
